# Q8.2

In [1]:
import numpy as np
from scipy import integrate

def LaplaceCoef(s, j, alpha):
    ''' Eq. (6.67) '''
    integrand = lambda x: np.cos(j*x)/(1 - 2*alpha*np.cos(x) + alpha**2)**s
    return (1/np.pi)*integrate.quad(integrand, 0, 2*np.pi)[0]

In [2]:
def Dbsj(s, j, alpha):
    ''' Eq. (6.70) '''
    LC1 = LaplaceCoef(s+1, j-1, alpha) # first Laplace coefficient 𝑏ₛ⁽ʲ⁾
    LC2 = LaplaceCoef(s+1, j, alpha)   # second...
    LC3 = LaplaceCoef(s+1, j+1, alpha) # third...
    return s*(LC1 - 2*alpha*LC2 + LC3)

In [3]:
def Inner_f_d(s, j, alpha):
    '''
    1st line in Table 8.1 (p. 330).
    For internal first-order resonance.
    '''
    term1 = LaplaceCoef(s, j, alpha)
    term2 = Dbsj(s, j, alpha)
    return -j*term1 - (alpha/2)*term2

In [4]:
# Test to verify first line of Table 8.5 (p. 334)
# 2:1 internal resonance w/ nominal 𝛼

alpha = 0.629961
af_d = alpha*Inner_f_d(.5, 2, alpha)
print('𝛼𝑓_d(𝛼) = {:.6f}'.format(af_d))

𝛼𝑓_d(𝛼) = -0.749966


In [5]:
def Outer_f_d(s, j, alpha):
    '''
    2nd line in Table 8.1 (p. 330).
    For external first-order resonance.
    '''
    term1 = LaplaceCoef(s, j-1, alpha)
    term2 = Dbsj(s, j-1, alpha)
    term3 = 0
    if j == 2: # for special case j = 2
        term3 = -2*alpha # absorbing 𝑓ₑ from Table 8.2 (see p. 349)
    return (3/2)*term1 + (alpha/2)*term2 + term3

In [6]:
def S(s, j, alpha):
    return (-3*2**(2/3)/Inner_f_d(s, j, alpha))**(1/3)

In [7]:
def Sprime(s, j, alpha):
    return (-3*j**2/Outer_f_d(s, j, alpha))**(1/3)

In [8]:
e = 0.001   # eccentricities of all three satellites
# satellite semi-major axes in units of planet radius
a1 = 7.31
a2 = 9.31
a3 = 11.6
a12 = a1/a2 # alpha 1-2
a13 = a1/a3 # alpha 1-3
a23 = a2/a3 # alpha 2-3

in_f_d12 = Inner_f_d(.5, 2, a12)
in_f_d13 = Inner_f_d(.5, 2, a13)
in_f_d23 = Inner_f_d(.5, 2, a23)
in_C12 = 2*S(.5, 2, a12)**(-3/2)/e
in_C13 = 2*S(.5, 2, a13)**(-3/2)/e
in_C23 = 2*S(.5, 2, a23)**(-3/2)/e

out_f_d12 = Outer_f_d(.5, 1, a12)
out_f_d13 = Outer_f_d(.5, 1, a13)
out_f_d23 = Outer_f_d(.5, 1, a23)
out_C12 = 2*Sprime(.5, 2, a12)**(-3/2)/e
out_C13 = 2*Sprime(.5, 2, a13)**(-3/2)/e
out_C23 = 2*Sprime(.5, 2, a23)**(-3/2)/e

print('𝛼₁₂ = {:.3f}'.format(a12))
print('𝛼𝑓_d(𝛼)₁₂ = {:.3f}'.format(a12*in_f_d12))
print('𝐶₁₂ = {:.0f}'.format(in_C12)) # 𝑏ₛ⁽ʲ⁾(𝛼)
print('𝐶₂₁ = {:.0f}'.format(out_C12))

print('\n𝛼₁₃ = {:.3f}'.format(a13))
print('𝛼𝑓_d(𝛼)₁₃ = {:.3f}'.format(a13*in_f_d13))
print('𝐶₁₃ = {:.0f}'.format(in_C23))
print('𝐶₃₁ = {:.0f}'.format(out_C13))

print('\n𝛼₂₃ = {:.3f}'.format(a23))
print('𝛼𝑓_d(𝛼)₂₃ = {:.3f}'.format(a23*in_f_d23))
print('𝐶₂₃ = {:.0f}'.format(in_C13))
print('𝐶₃₂ = {:.0f}'.format(out_C23))

𝛼₁₂ = 0.785
𝛼𝑓_d(𝛼)₁₂ = -1.905
𝐶₁₂ = 1428
𝐶₂₁ = 0-635j

𝛼₁₃ = 0.630
𝛼𝑓_d(𝛼)₁₃ = -0.751
𝐶₁₃ = 1493
𝐶₃₁ = 0-378j

𝛼₂₃ = 0.803
𝛼𝑓_d(𝛼)₂₃ = -2.131
𝐶₂₃ = 1000
𝐶₃₂ = 0-677j


# Q8.4

## Jovian External Resonances

### First-order
Generating associated $\alpha f_\textrm{d}(\alpha)$ terms, for the case of an external, $j : (j - 1)$ first-order resonance, needed in Eq. (8.32).  Here I use, as inputs for $\alpha$, the reciprocals of the nominal values given in Table 8.5.

In [9]:
def FOEfd(s, j, alpha):
    '''
    2nd line in Table 8.1 (see pp. 330, 349).
    For external first-order resonance for general j argument.
    '''
    prefac = 1/2
    term1 = -LaplaceCoef(s, j-1, alpha)
    term2 = 2*j*LaplaceCoef(s, j-1, alpha)
    term3 = alpha*Dbsj(s, j-1, alpha)
    term4 = 0
    if j == 2: # for special case j = 2 (see p. 349)
        term4 = -2*alpha*LaplaceCoef(s, j-1, alpha) # absorb 1st 𝑓ₑ from Table 8.2
    return prefac*(term1 + term2 + term3 + term4)

In [10]:
a12, a23, a34, a45 = 1/0.629961, 1/0.763143, 1/0.825482, 1/0.861774
af_d12 = a12*FOEfd(1/2, 2, a12)
af_d23 = a23*FOEfd(1/2, 3, a23)
af_d34 = a34*FOEfd(1/2, 4, a34)
af_d45 = a45*FOEfd(1/2, 5, a45)
print('𝑝 : 𝑝 + 𝑞\t 𝛼𝑓_d(𝛼)')
print('-------------------------')
print('1 : 2     \t{:.6f}'.format(af_d12))
print('2 : 3     \t {:.6f}'.format(af_d23))
print('3 : 4     \t {:.6f}'.format(af_d34))
print('4 : 5     \t {:.6f}'.format(af_d45))

𝑝 : 𝑝 + 𝑞	 𝛼𝑓_d(𝛼)
-------------------------
1 : 2     	-0.997620
2 : 3     	 0.294521
3 : 4     	 0.380772
4 : 5     	 0.466237


### Second-order
First we define a function to model the fifth entry in Table 8.1, corresponding to the $f_\textrm{d}$ expression for the case of an external, $j : (j - 2)$ second-order resonance.  For the $j = 3$ case, an indirect term given by the second entry in Table 8.2 must be absorbed into $f_\textrm{d}$.

In [11]:
def D2bsj(s, j, alpha):
    ''' Eq. (6.71) for the case n = 2 '''
    term1 = Dbsj(s+1, j-1, alpha)
    term2 = -2*alpha*Dbsj(s+1, j, alpha)
    term3 = Dbsj(s+1, j+1, alpha)
    term4 = -2*LaplaceCoef(s+1, j, alpha)
    return s*(term1 + term2 + term3 + term4)

In [12]:
def SOEfd(s, j, alpha):
    '''
    5th line in Table 8.1 (p. 330).
    For external second-order resonance for general j (p. 355)
    '''
    prefac = 1/8
    term1 = 2*LaplaceCoef(s, j-2, alpha)
    term2 = -7*j*LaplaceCoef(s, j-2, alpha)
    term3 = 4*j**2*LaplaceCoef(s, j-2, alpha)
    term4 = -2*alpha*Dbsj(s, j-2, alpha)
    term5 = 4*j*alpha*Dbsj(s, j-2, alpha)
    term6 = alpha**2*D2bsj(s, j-2, alpha)
    term7 = 0
    if j == 2: # for special case j = 2
        term7 = -(27/8)*alpha # absorbing 2nd 𝑓ₑ from Table 8.2 (see p. 356)
    return prefac*(term1 + term2 + term3 + term4 + term5 + term6) + term7

In [13]:
a35, a57, a79, a911 = 1/0.711379, 1/0.799064, 1/0.845740, 1/0.874782
af_d35 = a35*SOEfd(1/2, 5, a35)
af_d57 = a57*SOEfd(1/2, 7, a57)
af_d79 = a79*SOEfd(1/2, 9, a79)
af_d911 = a911*SOEfd(1/2, 11, a911)
print('𝑝 : 𝑝 + 𝑞\t 𝛼𝑓_d(𝛼)')
print('------------------------')
print('3 : 5     \t{:.6f}'.format(af_d35))
print('5 : 7     \t{:.6f}'.format(af_d57))
print('7 : 9     \t{:.5f}'.format(af_d79))
print('9 : 11    \t{:.5f}'.format(af_d911))

𝑝 : 𝑝 + 𝑞	 𝛼𝑓_d(𝛼)
------------------------
3 : 5     	0.430790
5 : 7     	0.801874
7 : 9     	1.28615
9 : 11    	1.88431


## Saturnian Internal Resonances

### First-order
Generating missing entries from Table 8.5 for the 7:6 and 8:7, first-order, internal resonance cases.

In [14]:
def FOIfd(s, j, alpha):
    '''
    1st line in Table 8.1 (p. 330).
    For internal first-order resonance.
    '''
    prefac = 1/2
    term1 = -2*j*LaplaceCoef(s, j, alpha)
    term2 = -alpha*Dbsj(s, j, alpha)
    return prefac*(term1 + term2)

In [15]:
a76, a87 = 8.60565/9.53707032, 8.72476/9.53707032
af_d76 = a76*FOIfd(.5, 7, a76)
af_d87 = a87*FOIfd(.5, 8, a87)
print('𝑝 + 𝑞 : 𝑝 \t 𝛼𝑓_d(𝛼)')
print('------------------------')
print('    7 : 6 \t{:.5f}'.format(af_d76))
print('    8 : 7 \t{:.5f}'.format(af_d87))

𝑝 + 𝑞 : 𝑝 	 𝛼𝑓_d(𝛼)
------------------------
    7 : 6 	-4.74741
    8 : 7 	-5.54886


### Second-order
We define a function to model the third entry in Table 8.1, corresponding to the $f_\textrm{d}$ expression for the case of an internal, $j : (j - 2)$ second-order resonance.

In [16]:
def SOIfd(s, j, alpha):
    '''
    3rd line in Table 8.1 (p. 330).
    For internal second-order resonances for general j (p. 355).
    '''
    prefac = 1/8
    term1 = -5*j*LaplaceCoef(s, j, alpha)
    term2 = 4*j**2*LaplaceCoef(s, j, alpha)
    term3 = -2*alpha*Dbsj(s, j, alpha)
    term4 = 4*j*alpha*Dbsj(s, j, alpha)
    term5 = alpha**2*D2bsj(s, j, alpha)
    return prefac*(term1 + term2 + term3 + term4 + term5)

In [17]:
# Tests to verify several lines of Table 8.5 (p. 334)
# second-rder, internal resonances w/ nominal 𝛼s

a31, a53, a119 = 0.480750, 0.711379, 0.874782
af_d31 = a31*SOIfd(1/2, 3, a31)
af_d53 = a53*SOIfd(1/2, 5, a53)
af_d119 = a119*SOIfd(1/2, 11, a119)
print('𝑝 + 𝑞 : 𝑝 \t  𝛼𝑓_d(𝛼)')
print('-------------------------')
print('    3 : 1 \t {:.6f}'.format(af_d31))
print('    5 : 3 \t {:.5f}'.format(af_d53))
print('   11 : 9 \t{:.4f}'.format(af_d119))

𝑝 + 𝑞 : 𝑝 	  𝛼𝑓_d(𝛼)
-------------------------
    3 : 1 	 0.287853
    5 : 3 	 2.32892
   11 : 9 	19.9639


In [18]:
a1311, a1513, a1715 = 8.53194/9.53707032, 8.66928/9.53707032, 8.77357/9.53707032
af_d1311 = a1311*SOIfd(1/2, 13, a1311)
af_d1513 = a1513*SOIfd(1/2, 15, a1513)
af_d1715 = a1715*SOIfd(1/2, 17, a1715)
print('𝑝 + 𝑞 : 𝑝 \t 𝛼𝑓_d(𝛼)')
print('------------------------')
print('   13 : 11\t{:.5f}'.format(af_d1311))
print('   15 : 13\t{:.5f}'.format(af_d1513))
print('   17 : 15\t{:.5f}'.format(af_d1715))

𝑝 + 𝑞 : 𝑝 	 𝛼𝑓_d(𝛼)
------------------------
   13 : 11	29.67873
   15 : 13	41.31212
   17 : 15	54.86324
